In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import random
from tqdm import trange
import pickle 
import shap
# import optuna
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, WhiteKernel, DotProduct, Matern
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [2]:
target_list = ['strength', 'modulus', 'elongation', 'tg']
x_dim = 6
bounds = np.array([[95., 120., 130., 120., 170., 120.],
                       [125., 300., 160., 300., 190., 300.]])
df = pd.read_csv('total_data.csv')
df = df.sample(frac=1, random_state=42)

X = np.array(df[['Temperature1/C', 'Time1/min', 'Temperature2/C','Time2/min', 'Temperature3/C', 'Time3/min']])
Y_total = np.array(df[['tensile strength/MPa','Young\'s modulus/Mpa', 'elongation_max/%', 'Tg']])

In [ ]:
with open('shap_states_1229.pth', 'rb') as f:
    state_dict = pickle.load(f)

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
colors = [(0, '#27e157'),  # 绿色
          (0.5, '#3F8BFF'),  # 蓝色
          (1, '#8A00B1')]  # 紫色
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)

# background-image: linear-gradient(to right, #27e157, #00dda1, #00d4d8, #00c6f5, #00b4f6, #46a6f6, #6f96ef, #9184e1, #bb6ccc, #dc50a9, #ed3479, #ec2b44);

for target in ['strength', 'modulus', 'elongation','tg']:

    test_x = state_dict[target]['data']
    shap_values = state_dict[target]['shap_values']
    # 可视化结果
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    # plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内

    font = {'family': 'Arial',
            'weight': 'bold',
            'size': 20}
    fig, ax = plt.subplots()
    x_name = ['T.1', 't.1', 'T.2', 't.2', 'T.3', 't.3']
    shap.summary_plot(shap_values, test_x, feature_names=x_name, max_display=30, show=False, cmap=cmap) 
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('black')
        spine.set_linewidth(1.5)
    for label in ax.get_yticklabels():
        label.set_fontsize(20)
        label.set_weight('bold')
        label.set_fontname("Arial")
        label.set_rotation(0)
    for label in ax.get_xticklabels():
        label.set_fontsize(20)
        label.set_weight('bold')
        label.set_fontname("Arial")
    ax.tick_params(axis='x', which='major', width=3)

    ax.xaxis.get_label().set_fontsize(20)
    ax.xaxis.get_label().set_fontweight('bold')
    ax.xaxis.get_label().set_fontname("Arial")
    plt.gcf().axes[-1].tick_params(labelsize=20)
    plt.gcf().axes[-1].set_ylabel("Feature value", fontdict=font)
    plt.gcf().set_size_inches(6, 4)
    plt.savefig('%s.png'%target, dpi=800, bbox_inches='tight')
    plt.show()

In [131]:
x_name = ['T1_0', 't1_1', 'T2_2', 't2_3', 'T3_4', 't3_5']
df = pd.DataFrame()

for key in ['strength', 'modulus', 'elongation', 'tg']:
    dict_ = state_dict[key]
    shap_values = dict_['shap_values']
    ip = np.abs(shap_values).mean(axis=0)
    ip_sorted = np.argsort(-ip)
    for i in range(len(x_name)):
        idx = ip_sorted[i]
        name = x_name[idx]
        df[key+'_x_'+name] = dict_['data'][:, idx]
        df[key+'_y_'+name] = shap_values[:, idx]
        

In [133]:
df.to_csv('shap_results.csv', index=True)